In [1]:
!git clone https://github.com/FaridRash/Advance-Statistical-Learnin--Semester-3

Cloning into 'Advance-Statistical-Learnin--Semester-3'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 39 (delta 7), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 33.01 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
data_path = "/kaggle/working/Advance-Statistical-Learnin--Semester-3/Dataset/EURUSD-2000-2020-15m.csv"
df = pd.read_csv(data_path)

In [4]:

def EMA(df, base, target, period, alpha=False):
    """
    Function to compute Exponential Moving Average (EMA)
    """
    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
    if (alpha == False):
        df[target] = con.ewm(span=period, adjust=False).mean()
    else:
        df[target] = con.ewm(alpha=alpha, adjust=False).mean()
    return df

df_ema = EMA(df, 'CLOSE', 'EMA_20', 20)

In [5]:
df_ema.head()

,DATE_TIME,HIGH,LOW,OPEN,CLOSE,EMA_20
0,2000.01.03 00:00:00,1.0080,1.0073,1.0073,1.0077,NaN
1,2000.01.03 00:15:00,1.0087,1.0076,1.0078,1.0086,NaN
2,2000.01.03 00:30:00,1.0089,1.0079,1.0087,1.0079,NaN
3,2000.01.03 00:45:00,1.0132,1.0078,1.0078,1.0128,NaN
4,2000.01.03 01:00:00,1.0133,1.0120,1.0129,1.0122,NaN


#Packing

In [6]:
def create_packs(df, pack_size):

    numerical_data = df[['HIGH', 'LOW', 'OPEN', 'CLOSE', 'EMA_20']].values
    
    num_packs = len(numerical_data) - pack_size + 1
    
    packs = np.array([numerical_data[i:i + pack_size] for i in range(num_packs)])
    
    return packs

In [7]:
pack_size = 100 
packs = create_packs(df_ema, pack_size)

In [8]:
type(packs), packs.shape

(numpy.ndarray, (500652, 100, 5))

#Normalization

In [9]:
def normalize_packs(packs):

    scaler = MinMaxScaler()
    
    num_packs, pack_size, num_features = packs.shape
    reshaped_packs = packs.reshape(-1, num_features)
    
    normalized_data = scaler.fit_transform(reshaped_packs)
    
    normalized_packs = normalized_data.reshape(num_packs, pack_size, num_features)
    
    return normalized_packs


In [10]:
normalized_packs = normalize_packs(packs)

print("Shape of normalized packs:", normalized_packs.shape)

Shape of normalized packs: (500652, 100, 5)


In [11]:
train_size = 0.7
val_size = 0.1
test_size = 0.2

packs_train, packs_temp, norm_packs_train, norm_packs_temp = train_test_split(
    packs, normalized_packs, test_size=(val_size + test_size), random_state=42)

In [12]:
val_ratio = val_size / (val_size + test_size)  # Adjust ratio for temp split
packs_val, packs_test, norm_packs_val, norm_packs_test = train_test_split(
    packs_temp, norm_packs_temp, test_size=(1 - val_ratio), random_state=42
)

In [13]:
print("Packs Train size:", len(packs_train))
print("Packs Validation size:", len(packs_val))
print("Packs Test size:", len(packs_test))
print('-----------')
print("Normalized Train size:", len(norm_packs_train))
print("Normalized Validation size:", len(norm_packs_val))
print("Normalized Test size:", len(norm_packs_test))

Packs Train size: 350456
Packs Validation size: 50065
Packs Test size: 100131
-----------
Normalized Train size: 350456
Normalized Validation size: 50065
Normalized Test size: 100131


In [14]:
def get_state(packs, index):

    state = packs[index]
    next_state = packs[index + 1] if index + 1 < len(packs) else None
    return state, next_state


In [15]:
def select_action(q_values, epsilon):

    if np.random.rand() < epsilon:
        action = np.random.choice([0, 1, 2])
    else:
        action = np.argmax(q_values)
    return action

In [16]:
def calculate_reward(action, open_price, stop_loss, target, target_multiplier, close_price, low, high, max_hold, current_hold):

    if action == 'buy':
        if low <= stop_loss: 
            return -1.0 
        elif high >= target:
            return target_multiplier

    elif action == 'sell':
        if high >= stop_loss: 
            return -1.0
        elif low <= target: 
            return target_multiplier

    elif current_hold >= max_hold:
        return (close_price - open_price) / open_price - 0.1 

    elif action == 'hold':
        return -0.05

    # Default: No action has triggered any event yet
    return 0.0